In [1]:
import numpy as np
import pandas as pd
import warnings
from datetime import datetime, timedelta
from matplotlib import pyplot as plt


warnings.filterwarnings('ignore')## Import Data## Import Data

For the baseline model, we calculate the reward function for each telescope on each day using $$f(D, R) = -\frac{1}{T(R)}\sum_{t = 1}^{T(R)}{\tau_{225}(D, t)},$$ where $D$ is the date we are looking at for the weather forecast, $R$ refers to the specific telescope, and $T(R)$ is the number of times forecasts being made in the telescope's observation timeframe. For each day, we calculate the day's reward by combining the telescopes' rewards in weighted average. $$F(D) = \sum_{i = 1}^N{W_{R_i}\times f(D, R_i)}.$$ Then we make decisions on whether to trigger the day based on the pure values of $F(D)$.

In the discounted model, we calculate $$F(D, r) = {\sum_{i = 1}^N{W_{R_i}\times f(D, R_i)}}\times{(1+r)^D},$$ where $r$ is the discount factor and then make decisions on $F(D,r)$. We multiply rather than divide here because the value is negative. We are going to experiment with different fixed value of $r$, and a function of $r$ depending on the variances in the forecasts, and compare the results with the ground-truth optimal path.

We also experimented with different weights. In the baseline model, we were using the output  file size as a measure of how important each telescope is. In the discounted model, we adjusted the weights by using the area of the telescope. We further scaled all $F(D, r)$ to be a weighted average of $f(D, R_i)$, by letting $W_{R_i}$ be the proportion of telescope $R_i$'s area in the total area. By doing this, we think the output can be more interpretable.

When we do evaluations, we are essentially looking backwards. Therefore, we directly calculate the scores without the discount factor $r$, for both the ground-truth path and our suggested path. The score given to any path $P$ is going to be $$S(P) = \frac{2}{1 + e^{- \sum_{j=1}^{\text{Days}}{F(D_{P,j})}}},$$ where $D_{P,j}$ is the $j$-th day selected by the path $P$. We have $2$ in the numerator, because the sigmoid function will give us values from 0 to 0.5 based on our negative $F$ values and we want to get scores from 0 to 1.


## Import Data

In [2]:
telescopes = ['12-meter','alma','apex','aste','iram','jcmt','lmt','sma','smt','spt']

In [3]:
starttime = datetime(2019,10,3,6)
endtime = datetime(2019,10,14,0) # not included
timestamps = np.arange(starttime, endtime, 
                       timedelta(hours=6)).astype(datetime)
databook = {}
for ts in telescopes:
    databook[ts] = dict.fromkeys(timestamps)

In [4]:
# for ts in telescopes:
#     for t in timestamps:
#         filepath = "data/"+ ts +"/"+ t.strftime("%Y%m%d_%H:%M:%S")
#         try:
#             df = pd.read_csv(filepath, delim_whitespace=True, skiprows = 1, header = None)
#             df.columns = ["date", "tau225", "Tb[k]", "pwv[mm]", "lwp[kg*m^-2]","iwp[kg*m^-2]","o3[DU]"]
#             df['date'] = pd.to_datetime(df['date'], format = "%Y%m%d_%H:%M:%S")
#             databook[ts][t] = df
#         except FileNotFoundError:
#             databook[ts][t] = None
# # databook is a dictionary of dictionaries of dataframes 
# # keys: telescope names
# # values: dictionaries of dataframes for one telescope
# # databook[telescope_name] is a dictionary of dataframes for one telescope
# # keys: timestamps when the forecast is made
# # values: forecast dataframe (None if missing)

In [5]:
################## fake data ###################
actual_time_span = np.arange(datetime(2019,7,1,0), datetime(2019,10,22,6), 
                       timedelta(hours=6)).astype(datetime)

i = 0
for site in telescopes:
    for t in timestamps:
        actual_time = actual_time_span[i]
        time_delta = t - actual_time
        filepath = "data/MaunaKea/"+ actual_time.strftime("%Y%m%d_%H:%M:%S")
        df = pd.read_csv(filepath, delim_whitespace=True, skiprows = 1, header = None)
        df.columns = ["date", "tau225", "Tb[k]", "pwv[mm]", "lwp[kg*m^-2]","iwp[kg*m^-2]","o3[DU]"]
        df['date'] = pd.to_datetime(df['date'], format = "%Y%m%d_%H:%M:%S") + time_delta
        databook[site][t] = df
        i += 1

In [6]:
print(telescopes[0],timestamps[1])
(databook[telescopes[0]][timestamps[1]]).head()

12-meter 2019-10-03 12:00:00


,date,tau225,Tb[k],pwv[mm],lwp[kg*m^-2],iwp[kg*m^-2],o3[DU]
0,2019-10-03 12:00:00,0.14078,39.414,3.2865,0.0,0.0,264.41
1,2019-10-03 13:00:00,0.14125,39.575,3.2760,0.0,0.0,264.04
2,2019-10-03 14:00:00,0.14093,39.556,3.2474,0.0,0.0,265.37
3,2019-10-03 15:00:00,0.13977,39.343,3.2021,0.0,0.0,265.87
4,2019-10-03 16:00:00,0.13965,39.350,3.1803,0.0,0.0,266.10


In [7]:
def day_reward(telescope_name, day_current_str, end_day_str, start_time, end_time, \
            use_as_evaluate = False):
    '''
    For the specified telescope, return a dataframe with two columns.
    The first column tells the day in the day window between 
        day_current_str and end_day_str (inclusive).
    The second column tells the average predicted tao225 given the day and the time window between
        start_time and end_time (inclusive).
    
    '''
    split_day_current = day_current_str.split('-')
    split_day_end = end_day_str.split('-') # include this day
    
    day_current = datetime(int(split_day_current[0]),int(split_day_current[1]),int(split_day_current[2]),0)
    day_end = datetime(int(split_day_end[0]),int(split_day_end[1]),int(split_day_end[2])+1,0)
    
    if not use_as_evaluate:
        mask = [t < day_current for t in databook[telescope_name]]
        t_valid = np.array([t for t in databook[telescope_name]])[mask]

        df_all = pd.concat([databook[telescope_name][t] for t in t_valid], axis =0)
    else:
        df_all = pd.concat([databook[telescope_name][t] for t in databook[telescope_name]], axis =0)
        
    df_tau_all = df_all.groupby('date').agg({'tau225':lambda x: list(x)}).reset_index()
    
    df_tau_all['latest'] = df_tau_all['tau225'].apply(lambda x: x[-1]) # baseline only use 
    
    
    df_tau_all = df_tau_all[(df_tau_all.date >= day_current) & (df_tau_all.date < day_end)]
    
    
    # calculate the reward for each day based on the schedule
    df_tau_all['day'] = df_tau_all.date.apply(lambda x: str(x).split(' ')[0])
    df_tau_all['time'] = df_tau_all.date.apply(lambda x: int(str(x).split(' ')[1][0:2]))
    
    df_tau_all = df_tau_all[(df_tau_all.time >= int(start_time)) & (df_tau_all.time <= int(end_time))]
    df_tau_day = pd.DataFrame(-df_tau_all.groupby('day')['latest'].mean())
    
    return df_tau_day

**2. Weighted sum the reward for each telescope according to the total Gbytes.** 
(so far we have not taken the telescopes '12-meter','aste','iram' into account as we haven't found corresponding schedule and weights)

In [8]:
# weight_telescope = [0, 22830.7, 26153.8, 0, 0, 12123.0, 22215.3, 12123.0, 18030.7, 26953.8]

# using the area (radius ** 2) of the telescope as weights 
weight_telescope = [12**2, 73**2, 12**2, 10**2, 30**2, 15**2, 32.5**2, 14.7**2, 10**2, 6**2]
schedule_telescope = [[0,1], [3,13], [3,15], [0,1], [0,1], [10,16], [6,16], [10,16], [8,16], [3,15]]


dict_schedule = dict(zip(telescopes, schedule_telescope))
dict_weight = dict(zip(telescopes, weight_telescope))

In [9]:
def all_day_reward(day_current_str, end_day_str):
    """
    calculate F(D) for D in range(day_current_str, end_day_str)
    taking in every single telescope we currently have
    weighted their f reward values
    based on area_i/total_area
    """
    # set up a dataframe
    telescopes_day_reward = day_reward(telescopes[0], day_current_str, end_day_str, \
                                       dict_schedule[telescopes[0]][0], dict_schedule[telescopes[0]][1]) \
                                       * dict_weight[telescopes[0]] 
    # 
    for i in telescopes[1:]:
        telescopes_day_reward += day_reward(i, day_current_str, end_day_str, \
                                            dict_schedule[i][0], dict_schedule[i][1])\
                                            * dict_weight[i] 
    return telescopes_day_reward / sum(weight_telescope)

In [10]:
exp = all_day_reward('2019-10-05', '2019-10-14')
exp

,latest
day,
2019-10-05,-0.141922
2019-10-06,-0.123334
2019-10-07,-0.147573
2019-10-08,-0.124194
2019-10-09,-0.244564
2019-10-10,-0.201999
2019-10-11,-0.192001
2019-10-12,-0.174121
2019-10-13,-0.107013


### Making Suggestions On-the-Go

In [11]:
def decision_making_single_discount_factor(day_current_str, end_day_str, days_to_trigger, discount = 0):
    # day_current_str: YYYY-MM-DD (str) (included)
    # end_day_str: YYYY-MM-DD (str) (included)
    # days_to_trigger: days to trigger (int)
    each_day_reward = all_day_reward(day_current_str, end_day_str)
    
    # inflate the values on each day
    a = np.array([n * ((1 + discount) ** i) for i, n in enumerate(each_day_reward['latest'])])
    
    # select the 'days_to_trigger' number of days having maximum reward values
    selected_days = np.array(each_day_reward.index)[np.argsort(a)[-1:-days_to_trigger-1:-1]]
    if day_current_str in selected_days:
        print('We suggest triggering on today {}'.format(day_current_str))
        output = True
    else: 
        print('We DO NOT suggest triggering on today {}'.format(day_current_str))
        output = False
    print('And we suggest to trigger by the following sequence: {}'.format(np.array(sorted(selected_days))))
    dic = dict(zip(each_day_reward.index, a))
    print('The discounted reward values for all the future days are ', dic)
    return output
          

## Evaluations

In [12]:
def simulate_single_discount_factor(days, num_days_trigger, discount):
    outputs = []
    for curr_day in days:
        days_left = int(num_days_trigger - np.sum(outputs))
        if days_left == 0:
            pass
        else:
            outputs.append(\
                           decision_making_single_discount_factor\
                           (curr_day, days[-1], days_left, discount))
        print("")
    return outputs

In [13]:
days = np.array(exp.index)
decisions = simulate_single_discount_factor(days, 5, 0)

We suggest triggering on today 2019-10-05
And we suggest to trigger by the following sequence: ['2019-10-05' '2019-10-06' '2019-10-08' '2019-10-13' '2019-10-14']
The discounted reward values for all the future days are  {'2019-10-05': -0.1419219538626451, '2019-10-06': -0.12333368289905193, '2019-10-07': -0.14757341884618774, '2019-10-08': -0.12419393363025463, '2019-10-09': -0.24456410305986243, '2019-10-10': -0.20199926409901492, '2019-10-11': -0.19200071191914517, '2019-10-12': -0.17412137823713936, '2019-10-13': -0.10701275264312339, '2019-10-14': -0.10240177702171466}

We DO NOT suggest triggering on today 2019-10-06
And we suggest to trigger by the following sequence: ['2019-10-07' '2019-10-08' '2019-10-09' '2019-10-13']
The discounted reward values for all the future days are  {'2019-10-06': -0.14313674958912342, '2019-10-07': -0.1303741316447293, '2019-10-08': -0.13293615815997786, '2019-10-09': -0.13367324239719797, '2019-10-10': -0.1839951780364977, '2019-10-11': -0.243018991

In [14]:
suggested_path = days[decisions + [False] * (len(days) - len(decisions))]
suggested_path

array(['2019-10-05', '2019-10-07', '2019-10-08', '2019-10-09',
       '2019-10-12'], dtype=object)

In [15]:
def best_path_afterwards(start_day_str, end_day_str, days_to_trigger, days_have_triggered = None):
    # start_day_str: YYYY-MM-DD (str) (included)
    # end_day_str: YYYY-MM-DD (str) (included)
    # days_to_trigger: days to trigger (int)
    # days_have_triggered: days acutally triggered (list of str)
    telescopes_day_reward = day_reward(telescopes[0], start_day_str, end_day_str, dict_schedule[telescopes[0]][0], dict_schedule[telescopes[0]][1], use_as_evaluate=True) * dict_weight[telescopes[0]]
    for i in telescopes[1:]:
        telescopes_day_reward += day_reward(i, start_day_str, end_day_str, dict_schedule[i][0], dict_schedule[i][1], use_as_evaluate=True) * dict_weight[i]
    telescopes_day_reward = telescopes_day_reward / sum(weight_telescope)
    
    all_path = telescopes_day_reward.sort_values(by='latest', ascending = False)
    best_path = all_path[:days_to_trigger]
    print('The best path to trigger based on ground-truth is {}'\
          .format(np.array(sorted(best_path.index))))
    print('The suggested path we predicted on the go is {}'\
         .format(days_have_triggered))
    print('The score given to the best path is {}'\
          .format(2 / (1 + np.exp(- best_path['latest'].sum()))))
    if days_have_triggered is not None:
        print('The score given to the suggested path is {}'\
              .format(2 / (1 + np.exp(- all_path.loc[days_have_triggered]['latest'].sum()))))
    return all_path

In [16]:
all_state = best_path_afterwards('2019-10-05', '2019-10-14', 5, days_have_triggered = suggested_path)


The best path to trigger based on ground-truth is ['2019-10-06' '2019-10-07' '2019-10-08' '2019-10-09' '2019-10-12']
The suggested path we predicted on the go is ['2019-10-05' '2019-10-07' '2019-10-08' '2019-10-09' '2019-10-12']
The score given to the best path is 0.676829584603156
The score given to the suggested path is 0.6572095047966228
